In [22]:
import pandas as pd
import pyterrier as pt
if not pt.java.started():
    pt.java.init()
from tqdm import tqdm


In [3]:
eval_dataset = pt.get_dataset(f'irds:msmarco-passage/dev')
corpus_df  = pd.DataFrame(eval_dataset.get_corpus_iter(verbose=True))

trec_file = "/mnt/datasets/cxj/exposure-fairness-extend/tctcolbert_msmarco-passage_dev_200.res"
trec_df = pd.read_csv(trec_file,sep=r"\s+", names=["qid", "Q0", "docno", "rank", "score", "run"], dtype={"qid": str, "docno": str})

msmarco-passage/dev documents: 100%|██████████| 8841823/8841823 [00:25<00:00, 346539.25it/s]


In [6]:
trec_df.head()

,qid,Q0,docno,rank,score,run
0,1048578,Q0,7187234,0,81.498650,tctcolbert_msmarco-passage_dev_200
1,1048578,Q0,4567130,1,81.423740,tctcolbert_msmarco-passage_dev_200
2,1048578,Q0,7187239,2,80.703580,tctcolbert_msmarco-passage_dev_200
3,1048578,Q0,2078221,3,80.142410,tctcolbert_msmarco-passage_dev_200
4,1048578,Q0,5365328,4,79.947495,tctcolbert_msmarco-passage_dev_200


In [32]:
result_df = trec_df.merge(corpus_df, how="left", on="docno")

In [33]:
result_df.head()

,qid,Q0,docno,rank,score,run,text
0,1048578,Q0,7187234,0,81.498650,tctcolbert_msmarco-passage_dev_200,Endless pools and swim spas are available in a...
1,1048578,Q0,4567130,1,81.423740,tctcolbert_msmarco-passage_dev_200,"$23,900. The 17' Endless Pool Spa features a l..."
2,1048578,Q0,7187239,2,80.703580,tctcolbert_msmarco-passage_dev_200,Endless Pools offers a family of swimming mach...
3,1048578,Q0,2078221,3,80.142410,tctcolbert_msmarco-passage_dev_200,1 A number of value brands will sell swim spas...
4,1048578,Q0,5365328,4,79.947495,tctcolbert_msmarco-passage_dev_200,"1 A top of the line, branded swim spa 9 feet b..."


In [ ]:
# model = pyterrier_dr.TctColBert('castorini/tct_colbert-v2-hnp-msmarco', batch_size=16, verbose=True)
# doc_vectors = model.encode_docs(tqdm(merged_src_df['text'].to_list()))
# merged_src_df['doc_vec'] = [vec for vec in doc_vectors]

In [14]:
from sentence_transformers import SentenceTransformer
import pandas as pd

model = SentenceTransformer("BAAI/bge-base-en-v1.5")

In [30]:
# result_df['doc_vec'] = model.encode(tqdm(result_df["text"].tolist())).tolist()

# vectors = []
# for text in tqdm(result_df["text"]):
#     vec = model.encode(text)
#     vectors.append(vec)
    
# result_df['doc_vec'] = vectors
# testdf = result_df[:1000].copy()
# testdf['doc_vec0'] = list(tqdm(model.encode(testdf["text"].tolist()),total=len(testdf)))


100%|██████████| 1000/1000 [00:00<00:00, 3026193.36it/s]


In [ ]:
from more_itertools import chunked

batch_size = 32
texts = result_df["text"].tolist()
vectors = []

# Total number of batches
num_batches = (len(texts) + batch_size - 1) // batch_size

for batch in tqdm(chunked(texts, batch_size), total=num_batches, desc="Batch encoding"):
    vecs = model.encode(batch)
    vectors.extend(vecs)
    
result_df['doc_vec'] = vectors

Batch encoding:  10%|▉         | 62516/631832 [2:14:23<19:00:41,  8.32it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Batch encoding:  24%|██▍       | 154399/631832 [5:29:57<15:03:52,  8.80it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Batch encoding:  25%|██▌       | 160148/631832 [5:42:40<16:52:43,  7.76it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To chan

In [ ]:
result_df.head()

In [ ]:
mmr = pyterrier_dr.MmrScorer(verbose=True)
diver_df = mmr.transform(result_df)
diver_df.to_csv('/mnt/datasets/cxj/exposure-fairness-extend/tctcolbert_msmarco-passage_dev_200_diver_df.csv', index=False)